In [1]:
import cv2
import numpy as np

In [2]:
# Web camera
cap = cv2.VideoCapture('video.mp4')

In [3]:
#min width reactangle
min_width_rect = 80

In [4]:
#min height reactangle
min_height_rect = 80

In [5]:
count_line_position = 550

In [6]:
# Initialize Substractor
#algo = cv2.bgsegm.createBackgroundSubtractorMOG()
#algo = cv2.createBackgroundSubtractorMOG2()
algo = cv2.createBackgroundSubtractorKNN()

In [7]:
def center_handle(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx,cy

In [8]:
detect = []
offset = 4 #allowable error between pixel
counter = 0

In [9]:
while True:
    ret,frame1 = cap.read()
    gray = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray,(3,3),5)
    #applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub,np.ones((5,5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5))
    dilatada = cv2.morphologyEx(dilat,cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada,cv2.MORPH_CLOSE, kernel)
    counterShape,h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    cv2.line(frame1,(25,count_line_position),(1200,count_line_position),(255,127,0),3)

    for (i,c) in enumerate(counterShape):
        (x,y,w,h) = cv2.boundingRect(c)
        validate_counter = (w>= min_width_rect) and (h>= min_height_rect)
        if not validate_counter:
            continue

        cv2.rectangle(frame1,(x,y),(x+w,y+h),(0,255,0),2)
        cv2.putText(frame1,"VEHICLE : "+str(counter),(x, y-20),cv2.FONT_HERSHEY_TRIPLEX,1,(255,244,0),2)

        center = center_handle(x,y,w,h)
        detect.append(center)
        cv2.circle(frame1,center,4, (0,0,255),-1)

        for (x,y) in detect:
            if y<(count_line_position+offset) and y>(count_line_position-offset):
                counter+=1
                cv2.line(frame1,(25,count_line_position),(1200,count_line_position),(0,127,255),3)
                detect.remove((x,y))
                print("vehicle counter : "+str(counter))


    cv2.putText(frame1,"VEHICLE COUNTER : "+str(counter),(450,70),cv2.FONT_HERSHEY_SIMPLEX,2,(0,0,255),5)



    #cv2.imshow('Detector',dilatada)
    cv2.imshow('Video original',frame1)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()

vehicle counter : 1
vehicle counter : 2
vehicle counter : 3
vehicle counter : 4
vehicle counter : 5
vehicle counter : 6
vehicle counter : 7
vehicle counter : 8
vehicle counter : 9
vehicle counter : 10


In [ ]:
def on_message(client, userdata, msg):
        """The callback for when a PUBLISH message is received from the server."""
        sl_no = 0
        date_time = time.strftime("%Y-%m-%d %H:%M:%S")
        print date_time
        dbConn = MySQLdb.connect("localhost", "database_name", "Password","database_name")
        print 'connected'
        with dbConn:
            cursor = dbConn.cursor()
            if msg.topic == "WHETHER MONITORING":
                print(msg.payload)
                payload = json.loads(msg.payload.decode('utf-8'))
                temperature = payload['temperature']
                humidity = payload['humidity']
                cursor.execute('INSERT INTO table_name VLUES (%s,%s,%s)',([sl_no], [date_time], [temperature], [humidity]))
                print'connected'
            dbConn.comit()
            cursor.close()